In [4]:
import pandas as pd
import os

# to save time, only decomment below 2 if necessary:
# from sentence_transformers import SentenceTransformer
# from sklearn.cluster import AgglomerativeClustering

from tqdm import tqdm
from datetime import datetime

In [2]:
df=pd.read_csv("r1.csv")

# below 1 is only for raw data processing:
# df[["CP", "Q", "CPCleaned", "QCleaned"]]=df[["CP", "Q", "CPCleaned", "QCleaned"]].applymap(lambda x : x.replace("  ", " ").lower().strip())

CPCleanedUnique=list(df.CPCleaned.unique())

In [5]:
# func def.:

def assign(q,orig:bool=False, cp="delete"):
    global df
    if orig==True:
        df.loc[df.Q==q,"CPCleaned"]=cp
    else:
        df.loc[df.QCleaned==q,"CPCleaned"]=cp
        
def getOriginalQ(cleanedQ):
    global df
    return df[df.QCleaned==cleanedQ].Q.value_counts()

def undo(combinedQ:str):
    global df
    df.loc[df.QCleaned==combinedQ,"QCleaned"]=df.loc[df.QCleaned==combinedQ,"Q"]

# decomment only needed:
# def groupSent(sentences, threshold=0.75):
#     model = SentenceTransformer('all-MiniLM-L6-v2')
#     embeddings = model.encode(sentences,show_progress_bar=False)
#     clustering = AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='average', distance_threshold=1-threshold)
#     clustering.fit(embeddings)
#     clusters = {}
#     for sentence_id, cluster_id in enumerate(clustering.labels_):
#         if cluster_id not in clusters:
#             clusters[cluster_id] = []
#         clusters[cluster_id].append(sentences[sentence_id])
#     return [group for group in clusters.values()]

genSubRule=lambda lol: {x: sorted(t, key=lambda x: len(x))[0] for t in lol for x in t}

def getQNum(qstr):
    global df
    return len(df[df.QCleaned==qstr])

def tabCountry(q):
    global df
    return df[df.QCleaned==q].Country.value_counts()

# with open("grpdQ.txt","r") as txt:
#     grpdQ=eval(txt.read())

def replace(qset, brief=None):
    '''list of questions, replace all with shortest/given'''
    qset=[x.strip() for x in qset]
    global df
    if brief==None:
        brief=sorted(qset,key=lambda x : len(x))[0]
    rplcidx=list(df[df["QCleaned"].isin(qset)].index)
    df.loc[rplcidx,"QCleaned"]=brief
    # df.loc[rplcidx,"cleaned"]=1

def getCountry(q):
    global df
    return df[df.QCleaned==q].Country.unique()

def CPVC(n):
    print(CPCleanedUnique[n])
    return df[df.CPCleaned==CPCleanedUnique[n]].QCleaned.value_counts()

def discard(q):
    global df
    df.drop(df[df.QCleaned==q].index, inplace=True)

def getYear(q):
    return df[df.QCleaned==q].AssesmentDate.str[-4:].unique()

def collapse(qcleaned, country):
    global df
    return df[(df.QCleaned==qcleaned) & (df.Country==country)].Q.value_counts()

def tab(cpnum=None,cp=None):
    assert cpnum!=None or cp!=None, "pass in at least 1 param."
    if cpnum!=None:
        cp=CPCleanedUnique[cpnum]
    print(cp)
    global df
    filtered_df=df[df['CPCleaned']==cp]
    return pd.crosstab(filtered_df['Country'], filtered_df['QCleaned'],margins=True).transpose()

def discardYear(qcleaned, country, year:int=2021):
    global df
    df.loc[(df.QCleaned==qcleaned) & (df.Country==country) & (df.AssesmentDate.str[-4:]==str(year)),"CPCleaned"]="delete"

def exportStata(path=datetime.now().strftime("%Y%m%d%H%M")):
    df.to_stata(path, version=118)



---
# grouping
*due to adding `delete` column, index reference in `CPCleanedUnique` might have changed.*

In [ ]:
# 12:
# assign("does the employer pay the required amount each year into the workers' participation fund?",orig=True)

replace([
    "does the employer collect and forward workers' contributions to social insurance funds on time?",
    "does the employer correctly calculate, collect, and forward workers' contributions to the social security corporation?",
    "forward social insurance",
    # which?:
    # "does the employer forward workers contributions to ona?",
    # "does the employer forward workers contributions to ofatma?"
])

In [22]:
tab(12)

social security and other benefits


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
are the workers' participation and welfare funds used and distributed as required?,584,0,0,0,0,0,0,584
collect and deposit workers' contributions into the fund,584,0,0,0,0,0,0,584
do workers receive required distributions from the workers' participation fund?,18,0,0,0,0,0,0,18
does the employer collect and forward workers' contributions to social insurance funds on time?,0,0,0,0,0,120,1802,1922
does the employer comply with other wage payments?,18,0,0,0,0,0,0,18
does the employer comply with requirements when workers have occupational accidents and diseases?,0,0,0,0,0,0,1802,1802
does the employer correctly calculate and send the employer contribution to the social security corporation?,0,0,0,0,474,0,0,474
"does the employer correctly calculate, collect, and forward workers' contributions to the social security corporation?",0,0,0,0,474,0,0,474
does the employer forward workers contributions to ofatma?,0,0,161,0,0,0,0,161


In [27]:
tab(13)

collective bargaining


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
cba at least as favorable as law,584,2503,161,851,0,122,1802,6023
employer consult with workers/union,584,2503,0,851,0,122,1802,5862
employer inform workers cba,584,2236,161,833,474,119,1802,6209
employer refused to bargain in good faith,584,2503,161,851,474,122,1802,6497
implement cba,584,2503,161,851,474,122,1802,6497
All,2920,12248,644,4237,1422,607,9010,31088


In [39]:
# 14(!):
# df.loc[df.Q=="does the accommodation have adequate eating and living areas?","QCleaned"]="does the accommodation have adequate eating and living areas?"

# replace([
#     "does the accommodation comply with minimum requirements?",
#     "does the accommodation provide adequate space?"
# ],"adequate space or minimum requirement")


## debug:
# for i in list(tab(14).index):
#     print(i)
#     print(getOriginalQ(i))
#     print("----------------------------------")

In [38]:
tab(14)

worker accommodation


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
adequate space or minimum requirement,584,3,161,833,474,119,1802,3976
are trained first aid personnel and adequate first aid supplies available in the accommodation?,0,0,0,0,474,0,0,474
does the accommodation have adequate cooking facilities?,584,3,161,833,474,0,0,2055
does the accommodation have adequate eating and living areas?,0,0,0,0,474,0,0,474
"does the accommodation have adequate toilets, showers, sewage and garbage disposal systems?",584,3,161,833,474,0,1802,3857
does the accommodation have enough safe water?,584,3,161,833,474,0,0,2055
does the accommodation have laundry facilities?,0,0,0,0,474,0,0,474
does the accommodation offer workers adequate privacy?,584,3,161,833,474,0,1802,3857
has the employer adequately prepared for emergencies in the accommodation?,584,2271,161,833,474,0,1802,6125


In [43]:
#15:
# replace([
#     "are wages paid on time?	",
#     "are workers paid on time, twice per month?",
#     "are workers paid within seven days from the date wages become payable?",
#     "does the employer pay wages on time and pay interest in case of late payment?	"
# ])

# replace([
#     "does the employer pay wages directly to workers in cash or by bank transfer?",
#     "are wages paid directly to workers?"
# ])

In [44]:
tab(15)

method of payment


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
are wages paid directly to workers?,566,2248,161,0,0,0,247,3222
are wages paid on time?,584,2294,161,851,474,122,1802,6288
are workers' cash wages paid in legal currency?,0,0,161,0,0,0,0,161
do in-kind wage payments comply with national law?,584,0,161,0,474,0,0,1219
All,1734,4542,644,851,948,122,2049,10890


In [ ]:
# 16-cleaned (6497):

In [45]:
tab(16)

forced labour and overtime


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
forced overtime under threat of penalty,584,2503,161,851,474,122,1802,6497
All,584,2503,161,851,474,122,1802,6497


In [ ]:
#17-OK:

In [46]:
tab(17)

interference and discrimination


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
employer disuades workers from unions,584,2503,161,833,474,119,0,4674
employer fires union official,584,2503,0,833,474,122,1802,6318
employer fires workers for union activity,584,2503,156,851,474,121,1802,6491
employer punishes workers for union activity,584,2503,161,839,474,122,1802,6485
employer threatens union members,584,2503,161,833,474,122,1802,6479
interfere union,566,2503,161,851,474,122,1802,6479
union membership/activity influences hiring,584,2503,161,833,474,121,1802,6478
workers are free to meet,584,2237,161,833,474,119,1802,6210
All,4654,19758,1122,6706,3792,968,12614,49614


In [48]:
#18
assign("does the employer restrict workers from leaving the workplace, dormitories or industrial zone? [differentiation question]")

In [49]:
tab(18)

coercion


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
are workers free to terminate their employment with reasonable notice?,584,2238,161,833,474,119,1802,6211
does the employer delay or withhold wage payments in order to coerce workers to work?,584,2238,161,833,474,117,1802,6209
"does the employer deny workers access to their personal documents (such as birth certificates, passports, work permits, id card.) when they need them?",584,2236,161,833,474,119,1802,6209
does the employer force workers to work to discipline them or as punishment for participation in a strike?,584,2236,161,833,474,119,1802,6209
does the employer require workers to work beyond the term of their contracts?,0,0,5,0,0,2,0,7
does the employer restrict workers' freedom to come and go from the dormitories and/or the industrial park or zone in which the factory is located?,584,2503,161,833,474,119,1802,6476
does the employer use any other coercive tactics to overwhelm workers' ability to make decision in their own interest?,584,0,161,833,0,2,0,1580
"does the employer use threats such as deportation, cancellation of visas or reporting to the authorities in order to force workers to stay at the job?",584,2236,161,833,474,119,1802,6209
does the employer use violence or the threat of violence to intimidate workers?,584,2236,161,833,474,119,1802,6209


In [60]:
#19:
# replace([
#     "does the employer dismiss workers for reasons that are no allowed by law?",
#     "does the employer dismiss workers for reasons that are not allowed by law?"
# ])

# replace([
#     "does the employer compensate workers for unused paid annual leave and unused earned compensatory rest day/s when they resign or are terminated?",
# "does the employer compensate workers for unused paid annual leave when they resign or are terminated?"
# ])

# replace([
#     "does the employer only terminate or dismiss workers for valid reasons?",
#     "does the employer only terminate workers for valid reasons?",
#     "does the employer terminate workers for reasons that are invalid under national law?"
# ])

# replace([
#     "does the employer notify the chief inspector and the collective bargaining agent before retrenching workers due to redundancy?",
#     "does the employer comply with legal requirements before laying off workers due to force majeure, and before retrenching workers due to redundancy?	"
# ])

# replace([
#     "does the employer compensate workers for unused paid annual leave and other legally required termination/resignation benefits?",
#     "does the employer compensate workers for unused paid annual leave when they resign or are terminated?"
# ])

# replace([
#     "do workers who resign or are terminated recieve severance based on years of service?",
#     "does the employer pay workers the correct severance pay?"
# ])

replace([
    "does the employer only terminate workers for valid reasons?",
    "does the employer dismiss workers for reasons that are no allowed by law?"
])

In [61]:
tab(19)

termination


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
are workers given the opportunities required under law to defend themselves before they are terminated?,584,0,0,833,474,119,1802,3812
do resigned or terminated workers receive all other termination benefits required by law?,0,0,0,0,0,0,1802,1802
does the employer compensate workers for unused paid annual leave when they resign or are terminated?,584,2503,161,851,474,0,1802,6375
does the employer comply with legal requirements before suspending workers due to changes in operations?,0,2503,156,0,474,122,1802,5057
does the employer get ministry of labour authorization prior to terminating workers for just cause?,0,0,0,0,0,122,0,122
does the employer notify the chief inspector and the collective bargaining agent before retrenching workers due to redundancy?,584,0,0,0,0,0,0,584
does the employer only terminate workers for valid reasons?,584,2503,161,851,474,0,1802,6375
does the employer pay all outstanding wages and indemnities within 48 hours of termination?,584,2315,0,0,0,0,1802,4701
does the employer pay terminated workers the correct damages when required?,0,2237,0,0,0,0,0,2237


In [62]:
#20-OK:

In [63]:
tab(20)

race and origin


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
"are workers harassed on the basis of their race, colour or origin?",584,2503,161,833,474,119,1802,6476
"do recruitment materials such as job announcements or job application forms refer to the applicant's race, colour or origin?",584,2503,161,0,474,119,1802,5643
"is an applicant's race, colour or origin a factor in hiring decisions?",584,2503,161,833,0,119,1802,6002
"is race, colour or origin a factor in decisions regarding conditions of work?",584,2503,161,0,0,119,1802,5169
"is race, colour or origin a factor in decisions regarding opportunities for promotion or access to training?",584,2503,161,833,474,119,1802,6476
"is race, colour or origin a factor in decisions regarding pay?",584,2503,161,0,474,119,1802,5643
"is race, colour or origin a factor in the employer's decisions regarding termination or retirement of workers?",584,2503,161,833,0,119,1802,6002
All,4088,17521,1127,3332,1896,833,12614,41411


In [69]:
#21:
# replace(["is an applicant's religion or political opinion a factor in hiring decisions?",
# "is an applicant's religion or political opinion referred to in recruitment materials such as job announcements or job application forms and/or a factor in the employer's decisions regarding hiring, termination or retirement of workers ?",
# "is applicant's religion or political opinion a factor in recruitment and hiring?"])

# replace([
# "is religion or political opinion a factor in decisions regarding opportunities for promotion or access to training?",
#     "is religion or political opinion a factor in decisions regarding conditions of work and/or opportunities for promotion or access to training?",
#     "is religion or political opinion a factor in decisions regarding conditions of work, access to training, promotion and pay?"
# ],"religion a factor of training or promotion")



In [70]:
tab(21)

religion and political opinion


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
are workers harassed on the basis of their religion or political opinion?,584,2503,161,833,474,119,1802,6476
do recruitment materials such as job announcements or job application forms refer to the applicant's religion or political opinion?,584,2503,161,0,0,119,1802,5169
is an applicant's religion or political opinion a factor in hiring decisions?,584,2503,161,833,474,119,1802,6476
is religion or political opinion a factor in decisions regarding conditions of work?,584,2503,161,0,0,119,1802,5169
is religion or political opinion a factor in decisions regarding pay?,584,2503,161,0,474,119,1802,5643
is religion or political opinion a factor in the employer's decisions regarding termination or retirement of workers?,584,2503,161,833,0,119,1802,6002
religion a factor of training or promotion,584,2503,161,833,474,119,1802,6476
All,4088,17521,1127,3332,1896,833,12614,41411


In [ ]:
#22-lack1:

In [71]:
tab(22)

freedom to associate


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
employer requires workers to join union,584,2503,161,851,474,122,1802,6497
freedom to form/join union,584,2503,161,851,474,124,27,4724
All,1168,5006,322,1702,948,246,1829,11221


In [72]:
#23-cleaned:

In [73]:
tab(23)

bonded labour


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
can workers who owe other types of debt to the employer freely leave their jobs?,584,2238,161,833,474,119,1802,6211
All,584,2238,161,833,474,119,1802,6211


In [80]:
#24:
# df.loc[df.Q=="do the employment contracts specify the terms and conditions of employment relation?","QCleaned"]=\
# "do the employment contracts specify the terms and conditions of employment relation?"

# replace([
#     "does the employer comply with requirements for migrant workers' contracts?",
#     "do the employment contracts comply with other legal requirements?"
# ])

# undo("has the employer provided all workers with a letter of appointment?")

In [86]:
tab(24)

employment contracts


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
"do all persons who perform work for the factory, both on the premises and offsite, have a contract?",0,2503,161,0,474,0,1802,4940
"do the contracts comply with the labour law, collective agreement and work rules?",0,0,161,1667,0,0,1802,3630
do the employment contracts comply with cambodian labour law and with the factory's internal regulations?,0,2286,0,0,0,0,0,2286
do the employment contracts comply with jordanian legal requirements?,0,0,0,0,474,0,0,474
do the employment contracts comply with other legal requirements?,0,2340,161,833,474,119,1802,5729
do the employment contracts specify the terms and conditions of employment relation?,0,0,0,0,0,121,0,121
do the factory bylaws comply with jordanian legal requirements and were they communicated to workers?,0,0,0,0,474,0,0,474
do the internal work rules comply with national law?,0,2265,161,0,0,119,1802,4347
"do the letters of appointment comply with the labour law, collective agreement and service rules?",584,0,0,0,0,0,0,584


In [87]:
df.to_csv("r2.csv", index=False)